# Battlefield System Explained

This notebook explains how the quantum battlefield simulation works.

## Overview

The battlefield is a grid where two teams (Quantum and Classical) fight using different movement strategies:
- **Quantum team**: Uses quantum algorithms to make optimal tactical decisions
- **Classical team**: Uses random movement

In [ ]:
import sys
sys.path.append('..')

from battlefield import QuantumBattlefield, Soldier
import quantum_library as qlib
import matplotlib.pyplot as plt

## 1. Creating a Battlefield

Battlefields are grids (default 4x4) where soldiers can move and fight.

In [ ]:
# Create a 4x4 battlefield
battlefield = QuantumBattlefield(width=4, height=4)

print(f"Battlefield size: {battlefield.width}x{battlefield.height}")
print(f"Total positions: {battlefield.board_size}")

## 2. Soldier Units

Each soldier has:
- **Position**: (x, y) coordinates
- **Strength**: Attack power (1-3)
- **Health**: Hit points (1-3)
- **Unit Type**: soldier, knight, or archer
- **Team**: Quantum or Classical
- **Range**: Attack range distance (1-3)

In [ ]:
# Create a soldier
soldier = Soldier(
    x=1,
    y=1,
    strength=2,
    health=3,
    unit_type='knight',
    team='Quantum',
    range_dist=1
)

print(f"Soldier at ({soldier.x}, {soldier.y})")
print(f"  Type: {soldier.unit_type}")
print(f"  Team: {soldier.team}")
print(f"  Stats: STR={soldier.strength}, HP={soldier.health}/{soldier.max_health}, RNG={soldier.range_dist}")

## 3. Initial Configurations

The battlefield supports three initial configurations:
1. **Face to Face** (default): Teams on opposite sides
2. **Surrounded**: One team in center, other on edges
3. **Random**: Completely random positions

In [ ]:
# Example: Face to Face configuration
battlefield_ftf = QuantumBattlefield(width=4, height=4)

# Define team compositions
quantum_comp = {
    'soldier': (2, 2, 2, 1),  # (count, strength, health, range)
    'archer': (1, 2, 1, 3)
}

classical_comp = {
    'knight': (2, 3, 3, 1),
    'soldier': (1, 2, 2, 1)
}

# Initialize with default (face to face)
battlefield_ftf.initialize_team('Quantum', quantum_comp, start_x_range=(0, 1))
battlefield_ftf.initialize_team('Classical', classical_comp, start_x_range=(2, 3))

print(f"Quantum soldiers: {sum(1 for s in battlefield_ftf.soldiers if s.team == 'Quantum')}")
print(f"Classical soldiers: {sum(1 for s in battlefield_ftf.soldiers if s.team == 'Classical')}")

In [ ]:
# Example: Surrounded configuration
battlefield_sur = QuantumBattlefield(width=4, height=4)

quantum_num = sum(count for count, _, _, _ in quantum_comp.values())
classical_num = sum(count for count, _, _, _ in classical_comp.values())

quantum_pos = battlefield_sur.get_initial_configuration_positions('surrounded', 'Quantum', quantum_num)
classical_pos = battlefield_sur.get_initial_configuration_positions('surrounded', 'Classical', classical_num)

battlefield_sur.initialize_team('Quantum', quantum_comp, positions=quantum_pos)
battlefield_sur.initialize_team('Classical', classical_comp, positions=classical_pos)

print("Surrounded configuration:")
print(f"  Quantum in center: {quantum_pos[:3]}...")
print(f"  Classical on edges: {classical_pos[:3]}...")

## 4. Battle Simulation

Each turn consists of:
1. **Move soldiers**: Quantum uses quantum algorithm, Classical uses random
2. **Resolve combat**: Soldiers in range attack each other
3. **Remove dead**: Soldiers with health ≤ 0 are removed
4. **Record history**: Track team sizes over time

In [ ]:
# Set quantum method (choose one)
qlib.set_quantum_method("exact")  # Fast for demo

# Run a complete battle
max_turns = 50
while not battlefield_ftf.is_battle_over() and battlefield_ftf.turn < max_turns:
    battlefield_ftf.step()

# Results
winner = battlefield_ftf.get_winner()
quantum_survivors, classical_survivors = battlefield_ftf.get_survivor_count()

print(f"Battle ended in {battlefield_ftf.turn} turns")
print(f"Winner: {winner}")
print(f"Survivors: Quantum={quantum_survivors}, Classical={classical_survivors}")

## 5. Movement Strategies

### Quantum Movement
1. Calculate H gradients in 4 directions
2. Build omega dict from gradients
3. Use quantum_step to find optimal move

### Classical Movement
- Random walk: dx, dy ∈ {-1, 0, 1}

In [ ]:
# Show movement logic
test_soldier = battlefield_ftf.soldiers[0] if battlefield_ftf.soldiers else soldier

print(f"Soldier at ({test_soldier.x}, {test_soldier.y})")
print(f"Team: {test_soldier.team}")

if test_soldier.team == "Quantum":
    enemies = [s for s in battlefield_ftf.soldiers if s.team != test_soldier.team and s.is_alive()]
    teammates = [s for s in battlefield_ftf.soldiers if s.team == test_soldier.team and s.is_alive() and s is not test_soldier]

    h_left = qlib.calculate_h_left(test_soldier, enemies, teammates)
    h_right = qlib.calculate_h_right(test_soldier, enemies, teammates)
    h_up = qlib.calculate_h_up(test_soldier, enemies, teammates)
    h_down = qlib.calculate_h_down(test_soldier, enemies, teammates)

    print(f"\nH gradients: Left={h_left}, Right={h_right}, Up={h_up}, Down={h_down}")
    print("Negative gradient = preferred direction")

## 6. Combat System

Combat rules:
- Soldiers can attack enemies within range
- Each soldier attacks the weakest enemy in range
- Damage = attacker's strength
- Soldiers die when health ≤ 0

In [ ]:
# Example combat
attacker = Soldier(x=1, y=1, strength=3, health=2, unit_type='knight', team='Quantum', range_dist=1)
defender = Soldier(x=2, y=1, strength=2, health=2, unit_type='soldier', team='Classical', range_dist=1)

print(f"Before attack:")
print(f"  Attacker: HP={attacker.health}")
print(f"  Defender: HP={defender.health}")

can_fight = attacker.can_fight(defender)
print(f"\nCan fight: {can_fight} (distance={attacker.distance_to(defender)}, range={attacker.range_dist})")

if can_fight:
    damage = attacker.attack(defender)
    print(f"\nDamage dealt: {damage}")
    print(f"Defender HP: {defender.health}")
    print(f"Defender alive: {defender.is_alive()}")

## 7. Visualizing Battles

The battlefield can be visualized showing unit positions and team history.

In [ ]:
# Plot battle history
if battlefield_ftf.history['turn']:
    plt.figure(figsize=(10, 4))

    plt.plot(battlefield_ftf.history['turn'], battlefield_ftf.history['team_a_count'],
             label='Quantum', color='blue', linewidth=2)
    plt.plot(battlefield_ftf.history['turn'], battlefield_ftf.history['team_b_count'],
             label='Classical', color='red', linewidth=2)

    plt.xlabel('Turn')
    plt.ylabel('Soldier Count')
    plt.title('Battle History')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("No battle history available (battle not run or already completed)")

## 8. Running Multiple Battles

For statistical analysis, run multiple battles with different configurations.

In [ ]:
# Run 10 quick battles
qlib.set_quantum_method("exact")  # Use fast method

results = []
for i in range(10):
    bf = QuantumBattlefield(width=4, height=4)
    bf.initialize_team('Quantum', quantum_comp, start_x_range=(0, 1))
    bf.initialize_team('Classical', classical_comp, start_x_range=(2, 3))

    while not bf.is_battle_over() and bf.turn < 50:
        bf.step()

    winner = bf.get_winner()
    results.append(winner)

# Statistics
quantum_wins = results.count('Quantum')
classical_wins = results.count('Classical')
draws = results.count(None)

print(f"Results from 10 battles:")
print(f"  Quantum: {quantum_wins} ({quantum_wins*10}%)")
print(f"  Classical: {classical_wins} ({classical_wins*10}%)")
print(f"  Draws: {draws} ({draws*10}%)")

## Summary

**Key Classes**:
- `Soldier`: Individual unit with stats and abilities
- `QuantumBattlefield`: Main simulation environment

**Key Methods**:
- `initialize_team()`: Set up teams with compositions
- `get_initial_configuration_positions()`: Get positions for different setups
- `step()`: Execute one turn of battle
- `is_battle_over()`: Check if battle has ended
- `get_winner()`: Determine the victor

**Initial Configurations**:
- Face to Face: Opposite sides (default)
- Surrounded: Center vs edges
- Random: Completely random

**Next Steps**:
- See `battlefield_tester_explained.ipynb` for automated testing
- See `quantum_library_explained.ipynb` for quantum algorithm details